In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 4"

In [2]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc

/home/tunguz/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
xgb.__version__

'1.6.0rc1'

In [4]:
!nvidia-smi

Thu Apr 21 08:13:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   34C    P0    59W / 275W |     14MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:47:00.0 Off |                    0 |
| N/A   

In [5]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM-80GB (UUID: GPU-a19c00c3-2832-fe38-1c43-c18db3e909da)
GPU 1: NVIDIA A100-SXM-80GB (UUID: GPU-58b97c92-e879-49d3-85b5-1d9615f10873)
GPU 2: NVIDIA A100-SXM-80GB (UUID: GPU-d21cfed4-2e1a-f313-839c-ea008aca027a)
GPU 3: NVIDIA DGX Display (UUID: GPU-bda05ff2-94b1-c885-24f7-586c8f86b408)
GPU 4: NVIDIA A100-SXM-80GB (UUID: GPU-e3b349d7-ac6c-77ab-3564-ed9d05d50bac)


In [6]:
#os.environ["CUDA_DEVICE_ORDER"]= "GPU-a5bb48d9-8574-a189-5dcf-39fe0abaef3e"

In [7]:
train_x = dd.read_csv('../../TPS_2021/input/tabular-playground-series-nov-2021/xgtrain.csv')
test_x = dd.read_csv('../../TPS_2021/input/tabular-playground-series-nov-2021/xgval.csv')

In [8]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [9]:
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=["GPU-a19c00c3-2832-fe38-1c43-c18db3e909da",
                                                 "GPU-58b97c92-e879-49d3-85b5-1d9615f10873",
                                                 "GPU-d21cfed4-2e1a-f313-839c-ea008aca027a",
                                                 "GPU-e3b349d7-ac6c-77ab-3564-ed9d05d50bac"])
client = Client(cluster)

2022-04-21 08:13:32,116 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-04-21 08:13:32,159 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-04-21 08:13:32,160 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-04-21 08:13:32,167 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [10]:
#cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES="GPU-a5bb48d9-8574-a189-5dcf-39fe0abaef3e")
#client = Client(cluster)

In [11]:
#cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=[0,1,2,4])
#client = Client(cluster)

In [12]:
!nvidia-smi

Thu Apr 21 08:13:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   34C    P0    62W / 275W |    428MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:47:00.0 Off |                    0 |
| N/A   

In [13]:
os.environ

environ{'SHELL': '/bin/bash',
        'CONDA_EXE': '/home/tunguz/anaconda3/bin/conda',
        '_CE_M': '',
        'PWD': '/home/tunguz',
        'LOGNAME': 'tunguz',
        'XDG_SESSION_TYPE': 'tty',
        'MOTD_SHOWN': 'pam',
        'HOME': '/home/tunguz',
        'LANG': 'en_US.UTF-8',
        'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.dz=01;31:*.gz=01;31:*.lrz=01;31:*.lz=01;31:*.lzo=01;31:*.xz=01;31:*.zst=01;31:*.tzst=01;31:*.bz2=01;31:*.bz=01;31:*.tbz=01;31:*.tbz2=01;31:*.tz=01;31:*.deb=01;31:*.rpm=01;31:*.jar=01;31:*.war=01;31:*.ear=01;31:*.sar=01;31:*.rar=01;31:*.alz=01;31:*.ace=01;31:*.zoo=01;31:*.cpio=01;31:*.7z=01;31:*.rz=01;31:*.cab=01;31:*.wim=01;31:*.swm=01;31:

In [14]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)

In [15]:
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)

In [16]:
num_round = 1000

In [17]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']), 
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    roc = roc_auc_score(test_y, predictions)
    
    return roc

In [18]:
study = optuna.create_study(direction='maximize')


[I 2022-04-21 08:13:52,205] A new study created in memory with name: no-name-b66533ae-cc0c-4e06-ab57-617774670ea0


In [19]:
%%time
study.optimize(objective, n_trials=2)

[08:13:56] task [xgboost.dask]:tcp://127.0.0.1:34553 got new rank 0
[08:13:56] task [xgboost.dask]:tcp://127.0.0.1:33099 got new rank 1
[08:13:56] task [xgboost.dask]:tcp://127.0.0.1:41975 got new rank 2
[08:13:56] task [xgboost.dask]:tcp://127.0.0.1:40519 got new rank 3
[I 2022-04-21 08:14:51,716] Trial 0 finished with value: 0.7460180830948313 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.0196718083368647, 'alpha': 1.3918270243416466, 'colsample_bytree': 0.3405273881990323, 'subsample': 0.8820340015298298, 'learning_rate': 0.017179573978527568, 'max_depth': 9, 'min_child_weight': 4, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.7460180830948313.
[08:14:51] task [xgboost.dask]:tcp://127.0.0.1:33099 got new rank 0
[08:14:51] task [xgboost.dask]:tcp://127.0.0.1:40519 got new rank 1
[08:14:51] task [xgboost.dask]:tcp://127.0.0.1:34553 got new rank 2
[08:14:51] task [xgboost.dask]:tcp://127.0.0.1:41975 got new rank 3
[I 2022-04-21 08:15

CPU times: user 10.3 s, sys: 6.12 s, total: 16.4 s
Wall time: 1min 13s


In [20]:
%%time
study.optimize(objective, n_trials=50)

[08:15:09] task [xgboost.dask]:tcp://127.0.0.1:41975 got new rank 0
[08:15:09] task [xgboost.dask]:tcp://127.0.0.1:33099 got new rank 1
[08:15:09] task [xgboost.dask]:tcp://127.0.0.1:34553 got new rank 2
[08:15:09] task [xgboost.dask]:tcp://127.0.0.1:40519 got new rank 3
[I 2022-04-21 08:16:34,768] Trial 2 finished with value: 0.7437520150854947 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.3244786966537936, 'alpha': 0.02348338839061649, 'colsample_bytree': 0.45813986230310666, 'subsample': 0.9834096396296961, 'learning_rate': 0.008305965154471701, 'max_depth': 15, 'min_child_weight': 185, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.7460180830948313.
[08:16:34] task [xgboost.dask]:tcp://127.0.0.1:33099 got new rank 0
[08:16:34] task [xgboost.dask]:tcp://127.0.0.1:40519 got new rank 1
[08:16:34] task [xgboost.dask]:tcp://127.0.0.1:34553 got new rank 2
[08:16:34] task [xgboost.dask]:tcp://127.0.0.1:41975 got new rank 3
[I 2022-04-21 

CPU times: user 1min 53s, sys: 45.6 s, total: 2min 38s
Wall time: 50min


In [20]:
%%time
study.optimize(objective, n_trials=100)

[10:19:46] task [xgboost.dask]:tcp://10.110.42.38:40591 got new rank 0
[10:19:46] task [xgboost.dask]:tcp://10.110.42.38:43553 got new rank 1
[10:19:46] task [xgboost.dask]:tcp://10.110.42.38:46159 got new rank 2
[I 2021-11-10 10:20:38,503] Trial 52 finished with value: 0.7478012808861039 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.005616442142668421, 'alpha': 5.086094935421799, 'colsample_bytree': 0.33470792936470767, 'subsample': 0.6900068262327083, 'learning_rate': 0.02012091481191884, 'max_depth': 13, 'min_child_weight': 174, 'eval_metric': 'logloss'}. Best is trial 31 with value: 0.7481132064706798.
[10:20:38] task [xgboost.dask]:tcp://10.110.42.38:43553 got new rank 0
[10:20:38] task [xgboost.dask]:tcp://10.110.42.38:40591 got new rank 1
[10:20:38] task [xgboost.dask]:tcp://10.110.42.38:46159 got new rank 2
[I 2021-11-10 10:21:35,849] Trial 53 finished with value: 0.7468275144140042 and parameters: {'objective': 'binary:logistic', 'tree

CPU times: user 4min 30s, sys: 1min 19s, total: 5min 50s
Wall time: 1h 16min 34s


In [21]:
%%time
study.optimize(objective, n_trials=250)

[11:36:21] task [xgboost.dask]:tcp://10.110.42.38:43553 got new rank 0
[11:36:21] task [xgboost.dask]:tcp://10.110.42.38:46159 got new rank 1
[11:36:21] task [xgboost.dask]:tcp://10.110.42.38:40591 got new rank 2
[I 2021-11-10 11:37:17,725] Trial 152 finished with value: 0.7478432167330396 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.006105193376215936, 'alpha': 4.185287484968151, 'colsample_bytree': 0.328747151910861, 'subsample': 0.6863522101754208, 'learning_rate': 0.02386060941733218, 'max_depth': 17, 'min_child_weight': 217, 'eval_metric': 'logloss'}. Best is trial 103 with value: 0.748210163763588.
[11:37:17] task [xgboost.dask]:tcp://10.110.42.38:46159 got new rank 0
[11:37:17] task [xgboost.dask]:tcp://10.110.42.38:40591 got new rank 1
[11:37:17] task [xgboost.dask]:tcp://10.110.42.38:43553 got new rank 2
[I 2021-11-10 11:38:12,318] Trial 153 finished with value: 0.7478373683888017 and parameters: {'objective': 'binary:logistic', 'tree

CPU times: user 10min 49s, sys: 3min 12s, total: 14min 1s
Wall time: 2h 58min 24s


In [23]:
study.best_trial.params

{'objective': 'binary:logistic',
 'tree_method': 'gpu_hist',
 'lambda': 0.007732898120949669,
 'alpha': 2.051267157385725,
 'colsample_bytree': 0.31569537175581425,
 'subsample': 0.810515453664132,
 'learning_rate': 0.025364136750997607,
 'max_depth': 13,
 'min_child_weight': 239,
 'eval_metric': 'logloss'}